/* Run in Google Collab */

In [ ]:
!wget https://www.dropbox.com/s/w3zlhing4dkgeyb/train.zip?dl=0

!unzip train.zip?dl=0

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.models import Model
from keras.layers import Flatten, Dense

from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

## Model Building

In [ ]:
base_model = MobileNet(input_shape=(224,224,3), include_top=False)

# To prevent model from retraining

for layer in base_model.layers:
  layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(units=7, activation='softmax')(x)

model = Model(base_model.input, x)

In [ ]:
# Layers of the model

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Preparing Data Using Data Generator

In [ ]:
train_dataG = ImageDataGenerator(
    zoom_range = 0.2,
    shear_range = 0.2,
    horizontal_flip =True,
    rescale = 1./255
)

train_data = train_dataG.flow_from_directory(directory = "/content/train",
                                             target_size = (224,224),
                                             batch_size = 32)

# Batch size indicates how many cycles it will run

train_data.class_indices

## Validating the Data

In [ ]:
val_dataG = ImageDataGenerator(rescale = 1/255)

val_data = val_dataG.flow_from_directory(directory = "/content/train",
                                         target_size=(224,224),
                                         batch_size=32)

## Visualizing the Data

In [ ]:
# Check the current backend, should be TkAgg
print("Matplotlib Backend:", plt.get_backend())

In [ ]:
# Visualizing images

t_img, label = train_data.next()

# Function for plotting images

def plotImages(img_arr, label):

  count = 0
  for im, l in zip(img_arr, label):
    plt.imshow(im)
    plt.title(im.shape)
    plt.axis = False
    plt.show()

    count += 1
    if count == 10:
      break

plotImages(t_img, label)

## Model Checkpoints and Early Stopping

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# For Early Stopping 
es = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=5, verbose=1, mode='auto')

# For Model Checkpoint
mc = ModelCheckpoint(filepath="best_model.h5", monitor='val_accuracy', verbose=1, save_best_only = True, mode='auto')

call_back = [es, mc]

In [ ]:
hist = model.fit_generator(train_data, steps_per_epoch=10, epochs=30, validation_data = val_data, 
                           validation_steps=8,callbacks=[es,mc])

In [ ]:
# Loading best fit model

from keras.models import load_model
model = load_model("/content/best_model.h5")

In [ ]:
h = hist.history
h.keys()

## Plotting Accuracy

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c = 'red')
plt.title("accuracy vs validated accuracy")
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'], c = 'red')
plt.title("loss vs validated loss")
plt.show()

In [ ]:
# Map op values

op = dict(zip(train_data.class_indices.values(), train_data.class_indices.keys()))

## Checking Class of uploaded image

In [ ]:
# Input/Upload any image from your desktop

path = '/content/image.jpg'
img = load_img(path, target_size=(224,224))

i = img_to_array(img)/255
input_arr = np.array([i])
input_arr.shape

predict = np.argmax(model.predict(input_arr))

print(f" Image is of {op[predict]}")

# Display the image
plt.imshow(input_arr[0])
plt.title("Uploaded Image")
plt.show()